<a href="https://colab.research.google.com/github/ShivamCholin/FeedForward_NeuralNetwork/blob/main/Wand_COnfusion_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random          
import numpy as np      
from time import time    
import math

In [2]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))
def dsigmoid(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [3]:
def relu(z):
    return np.maximum(z, 0)
def drelu(z):
    return np.heaviside(z, 1)

In [4]:
def tanh(z):
    return np.tanh(z)
def dtanh(z):
    return 1-tanh(z)**2

In [5]:
def softmax(x):
		e_x = np.exp(x)
		return e_x / np.sum(e_x)

In [6]:
def sqd_loss(nn, te_data):
  loss=0
  for x,y in te_data:
    act=forward(nn,x)
    act=softmax(act)
    loss+=np.sum((act-y)**2)
  return loss/len(te_data)

In [7]:
def ce_loss(nn, te_data):
  loss=0
  for x,y in te_data:
    act=forward(nn,x)
    act=softmax(act)
    loss+=(-np.log(max(act))+(nn.lmbda/2)*np.sum([np.sum(w**2) for w in nn.weights]))[0]
  return loss/len(te_data)

In [8]:
def sqdcost_d(act,y,s):
    act=softmax(act)
    ret=[]
    ll=len(act)
    for j in range(ll):
      add=0
      for i in range(ll):
        add+=2*(act[i]-y[i])*act[i]*((i==j)-act[j])
      ret.append(add)
    return np.array(ret)/s

In [9]:
def cecost_d(act, y,s):
    act=softmax(act)
    act= act-y
    return act

In [10]:
def backward(nn, x, y):
    m = x.shape[1]
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    activation = x 
    acts = [x]
    zs = []    
    for b, w in zip(nn.biases, nn.weights):
        z = np.dot(w, activation) + b 
        zs.append(z)        
        activation = nn.act(z)   
        acts.append(activation)
    delta = nn.cost(acts[-1], y,nn.batch_size) * nn.dact(zs[-1])
    nb[-1] = delta
    nw[-1] = np.dot(delta, acts[-2].transpose()) + nn.lmbda * nn.weights[-1]
    for i in range(2, nn.num_layers):
        z = zs[-i]
        sp = nn.dact(z)
        delta = np.dot(nn.weights[-i + 1].transpose(), delta) * sp
        nb[-i] = delta
        nw[-i] = np.dot(delta, acts[-i - 1].transpose())+ nn.lmbda * nn.weights[-i]
    return (nb, nw)

In [11]:
def nagbackward(nn, x, y):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    activation = x 
    acts = [x]
    zs = []  
    new_weights = [w-v*nn.gamma for w,v in zip(nn.weights,nn.vw)]
    new_biases = [b-v*nn.gamma for b,v in zip(nn.biases,nn.vb)]
    for b, w in zip(new_biases, new_weights):
        z = np.dot(w, activation) + b 
        zs.append(z)        
        activation = nn.act(z)   
        acts.append(activation)
    delta = nn.cost(acts[-1], y,nn.batch_size) * nn.dact(zs[-1])
    nb[-1] = delta
    nw[-1] = np.dot(delta, acts[-2].transpose())+ nn.lmbda * nn.weights[-1]
    for i in range(2, nn.num_layers):
        z = zs[-i]
        sp = nn.dact(z)
        delta = np.dot(new_weights[-i + 1].transpose(), delta) * sp
        nb[-i] = delta
        nw[-i] = np.dot(delta, acts[-i - 1].transpose())+ nn.lmbda * nn.weights[-i]
    return (nb, nw)

In [12]:
def learn(nn, tr_data,te_data,va_data):
    n = len(tr_data)
    for j in range(nn.epochs):
        time_start=time()
        random.shuffle(tr_data)
        batches = [tr_data[k: k + nn.batch_size] for k in range(0,n,nn.batch_size)]
        for batch in batches:
            nn.grad(nn, batch)
        time_end=time()
        print('Epoch {0}:time taken {1} seconds, accuracy {2} % , loss {3}'.format(f'{j + 1:2}',1.0*time_end-time_start, 100.0 * evaluate(nn, tr_data) / len(tr_data), nn.loss(nn,tr_data)))

In [13]:
def forward(nn, a):
    for b, w in zip(nn.biases, nn.weights):
        a = nn.act(np.dot(w, a) + b)
    return a

In [14]:
def sgd(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    num=1
    for x, y in batch:
        dnb, dnw = backward(nn, x, y)
        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]
    nn.weights = [w - (nn.learning_rate ) * nw for w, nw in zip(nn.weights, nw)]
    nn.biases  = [b - (nn.learning_rate) * nb for b, nb in zip(nn.biases, nb)]

In [15]:
def mbgd(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]

    nn.vw = [vweight*nn.gamma + nn.learning_rate * nw for vweight,nw in zip(nn.vw,nw)]
    nn.vb = [vbiases*nn.gamma + nn.learning_rate * nb for vbiases,nb in zip(nn.vb,nb)]
    nn.weights = [w - v for w, v in zip(nn.weights, nn.vw)]
    nn.biases  = [b - v for b, v in zip(nn.biases, nn.vb)]

In [16]:
def nag(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = nagbackward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]

    nn.vw = [vweight*nn.gamma + nn.learning_rate * nw for vweight,nw in zip(nn.vw,nw)]
    nn.vb = [vbiases*nn.gamma + nn.learning_rate * nb for vbiases,nb in zip(nn.vb,nb)]
    nn.weights = [w - v for w, v in zip(nn.weights, nn.vw)]
    nn.biases  = [b - v for b, v in zip(nn.biases, nn.vb)]

In [17]:
def rmsprop(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]

    nn.gew = [gew*nn.gamma + (1.0-nn.gamma)*np.square(nw) for gew,nw in zip(nn.gew,nw)]
    nn.geb = [geb*nn.gamma + (1.0-nn.gamma)*np.square(nb) for geb,nb in zip(nn.geb,nb)]
    nn.weights = [w - nn.learning_rate * nw / np.sqrt(ge+nn.epsilon) for w, ge,nw in zip(nn.weights, nn.gew,nw)]
    nn.biases  = [b - nn.learning_rate * nb / np.sqrt(ge+nn.epsilon) for b, ge,nb in zip(nn.biases, nn.geb,nb)]

In [18]:
def adam(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 
        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]

    nn.mw = [nn.beta1w*m + (1.0-nn.beta1w)*nw for m,nw in zip(nn.mw,nw)]
    nn.mb = [nn.beta1b*m + (1.0-nn.beta1b)*nb for m,nb in zip(nn.mb,nb)]
    nn.vw = [nn.beta2w*v + (1.0-nn.beta2w)*np.square(nw) for v,nw in zip(nn.vw,nw)]
    nn.vb = [nn.beta2b*v + (1.0-nn.beta2b)*np.square(nb) for v,nb in zip(nn.vb,nb)]
    nn.beta1_expw *= nn.beta1w
    nn.beta2_expw *= nn.beta2w
    nn.beta1_expb *= nn.beta1b
    nn.beta2_expb *= nn.beta2b
    nn.weights = [w - nn.learning_rate *(m/(1.0-nn.beta1_expw))/ (np.sqrt(v / (1.0 - nn.beta2_expw)) + nn.epsilon) for w, m,v in zip(nn.weights, nn.mw,nn.vw)]
    nn.biases = [b - nn.learning_rate *(m/(1.0-nn.beta1_expb))/ (np.sqrt(v / (1.0 - nn.beta2_expb)) + nn.epsilon) for b, m,v in zip(nn.biases, nn.mb,nn.vb)]

In [19]:
def nadam(nn, batch):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]


    nn.mw = [nn.beta1w*m + (1.0-nn.beta1w)*nw for m,nw in zip(nn.mw,nw)]
    nn.mb = [nn.beta1b*m + (1.0-nn.beta1b)*nb for m,nb in zip(nn.mb,nb)]
    nn.vw = [nn.beta2w*v + (1.0-nn.beta2w)*np.square(nw) for v,nw in zip(nn.vw,nw)]
    nn.vb = [nn.beta2b*v + (1.0-nn.beta2b)*np.square(nb) for v,nb in zip(nn.vb,nb)]
    #nn.weights = [w - (eta ) * nw for w, nw in zip(nn.weights, nw)]
    #nn.biases  = [b - (eta ) * nb for b, nb in zip(nn.biases, nb)]
    nn.beta1_expw *= nn.beta1w
    nn.beta2_expw *= nn.beta2w
    nn.beta1_expb *= nn.beta1b
    nn.beta2_expb *= nn.beta2b
    mwn=[m/(1-nn.beta1_expw) for m in nn.mw]
    vwn=[v/(1-nn.beta2_expw) for v in nn.vw]
    mbn=[m/(1-nn.beta1_expb) for m in nn.mb]
    vbn=[v/(1-nn.beta2_expb) for v in nn.vb]
    mwnn=[nn.beta1w * m + ((1-nn.beta1w)/(1-nn.beta1_expw))*g for m,g in zip(mwn,nw)]
    mbnn=[nn.beta1b * m + ((1-nn.beta1b)/(1-nn.beta1_expb))*g for m,g in zip(mbn,nb)]

    nn.weights = [w - nn.learning_rate *m/ (np.sqrt(v) + nn.epsilon) for w, m,v in zip(nn.weights, mwnn,vwn)]
    nn.biases = [b - nn.learning_rate *m/ (np.sqrt(v)+ nn.epsilon) for b, m,v in zip(nn.biases, mbnn,vbn)]

In [20]:
class Network:
    num_layers=0
    biases=[]
    weights=[]
    def __init__(self,nl,x,y,act1,act2,lmbda,g_func,c_func,l_func,lr,bs,e):
      self.num_layers=nl
      self.biases=x
      self.weights=y
      self.act=act1
      self.dact=act2
      self.grad=g_func
      self.vw=[np.zeros(w.shape) for w in self.weights]
      self.vb=[np.zeros(b.shape) for b in self.biases]
      self.mw=[np.zeros(w.shape) for w in self.weights]
      self.mb=[np.zeros(b.shape) for b in self.biases]
      self.gew=[np.zeros(w.shape) for w in self.weights]
      self.geb=[np.zeros(b.shape) for b in self.biases]
      self.gamma=0.9
      self.epsilon = 1e-8
      self.beta1w = 0.9
      self.beta2w = 0.999
      self.beta1_expw = 1.0
      self.beta2_expw = 1.0
      self.beta1b = 0.9
      self.beta2b = 0.999
      self.beta1_expb = 1.0
      self.beta2_expb = 1.0
      self.lmbda=lmbda
      self.cost=c_func
      self.learning_rate=lr
      self.epochs=e
      self.batch_size=bs
      self.loss=l_func

def init_network(layers,actfunc,lmbda,grad_function,Loss_f,lr,bs,e,xav):
    if xav:
      weights=[np.random.randn(y, x)*math.sqrt(6/(x+y)) for x, y in zip(layers[:-1], layers[1:])]
    else:
      weights=[np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])]
    biases = [np.random.randn(y, 1) for y in layers[1:]]
    if actfunc=="tanh":
      act1=tanh
      act2=dtanh
    elif actfunc=="sigmoid":
      act1=sigmoid
      act2=dsigmoid
    else:
      act1=relu
      act2=drelu
    if grad_function=="SGD":
      g_func=sgd
    elif grad_function=="MBGD":
      g_func=mbgd
    elif grad_function=="NAG":
      g_func=nag
    elif grad_function=="RMSPROP":
      g_func=rmsprop
    elif grad_function=="ADAM":
      g_func=adam
    elif grad_function=="NADAM":
      g_func=nadam
    if Loss_f=="MSE":
      cost_func=sqdcost_d
      loss_func=sqd_loss
    else:
      cost_func=cecost_d
      loss_func=ce_loss
    return Network(len(layers),biases,weights,act1,act2,lmbda,g_func,cost_func,loss_func,lr,bs,e)

In [21]:
def evaluate(nn, te_data):
    test_results = [(np.argmax(forward(nn, x)), y) for (x, y) in te_data]
    return sum(int(x == np.argmax(y)) for (x, y) in test_results)

In [36]:
def confusion_evaluate(nn, te_data):
    test_results = [(forward(nn, x)) for x in te_data]
    return test_results

In [23]:
import keras
def change_y(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e
fashion_mnist = keras.datasets.fashion_mnist
tr_data, te_data = fashion_mnist.load_data()
training_x = [np.reshape(x, (784, 1))/255 for x in tr_data[0]]
training_y = [change_y(y) for y in tr_data[1]]
tr_data = zip(training_x, training_y)
test_inputs = [np.reshape(x, (784, 1))/255 for x in te_data[0]]
test_y = [change_y(y) for y in te_data[1]]
te_data = zip(test_inputs, test_y)
tr_data=list(tr_data)
va_data=tr_data[0:10000]
tr_data=tr_data[10000:len(tr_data)]
te_data=list(te_data)

4423680/4422102 [==============================] - 0s 0us/step


In [27]:

no_of_hlayers=5
size_of_each_hlayer=64
Activation_function="tanh"
L2_regularisation=0
Gradient_descent_optimizer="NADAM"
epochs = 15
batch_size =64
learning_rate = 0.0001
Xavier=1
loss_function = "Cross_entropy"
L=[]
L.append(784)
for _ in range(no_of_hlayers):
  L.append(size_of_each_hlayer)
L.append(10)
nn = init_network(L,Activation_function,L2_regularisation,Gradient_descent_optimizer,loss_function,learning_rate,batch_size,epochs,Xavier)
print('start')
learn(nn, tr_data,te_data,va_data)
print('accuracy {0}%'.format(100.0 * evaluate(nn, te_data) / len(te_data)))
  

start
Epoch  1:time taken 46.27036714553833 seconds, accuracy 73.0 % , loss 1.05110813680282
Epoch  2:time taken 46.49517774581909 seconds, accuracy 81.562 % , loss 0.9650121836682377
Epoch  3:time taken 46.22844219207764 seconds, accuracy 83.574 % , loss 0.9322050992573321
Epoch  4:time taken 46.44514012336731 seconds, accuracy 84.748 % , loss 0.9160075281023722
Epoch  5:time taken 46.48725867271423 seconds, accuracy 85.456 % , loss 0.9009809086347966
Epoch  6:time taken 46.81849956512451 seconds, accuracy 85.892 % , loss 0.89687220184306
Epoch  7:time taken 46.327372312545776 seconds, accuracy 86.47 % , loss 0.8988644131501226
Epoch  8:time taken 46.5325562953949 seconds, accuracy 86.8 % , loss 0.8918344077394144
Epoch  9:time taken 46.251909017562866 seconds, accuracy 87.166 % , loss 0.8938141600587549
Epoch 10:time taken 46.28386116027832 seconds, accuracy 87.26 % , loss 0.8891149103222319
Epoch 11:time taken 46.48797082901001 seconds, accuracy 87.682 % , loss 0.8847732723281128
Ep

In [28]:
!pip install wandb

     |████████████████████████████████| 2.0MB 5.6MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 163kB 28.4MB/s 
     |████████████████████████████████| 133kB 26.2MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=7960b6d65794ec132f8c6ff6f2b845d65e12554ec40abacb33fe65ee91e2434a
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=a26ebf0c81fcc21f5907420c1b4824e2361134bf9e84f44be8b055c7b41d1eef
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [29]:
import wandb

In [39]:
x=test_inputs
y=test_y
val_predictions = confusion_evaluate(nn,x)
top_pred_ids = [np.argmax(x) for x in val_predictions]
ground_truth_ids = [np.argmax(x) for x in y]
wandb.init(project='assignemnt-1-cs6910', entity='cs20m061',name="confusion matrix generation")
wandb.log({"confusion matrix" : wandb.plot.confusion_matrix(
            preds=top_pred_ids, y_true=ground_truth_ids,
            class_names=['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot'])})

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [40]:
wandb.finish()

_runtime,3
_timestamp,1615634665
_step,0


_runtime,▁
_timestamp,▁
_step,▁
